In [0]:
--bronze table

CREATE OR REFRESH STREAMING TABLE status_bronze
  COMMENT "Ingest raw data from cloud to bronze"
  TBLPROPERTIES (
    "quality" = "bronze",
    "pipeline.reset.allowed" = false
  )
AS
SELECT
  *,
  current_timestamp() AS processing_time,
  _metadata.file_name AS source_file
FROM STREAM read_files(
  "${source}/status",
  format => "JSON"
);

-- silver table

CREATE OR REFRESH STREAMING TABLE status_silver
  (
    CONSTRAINT valid_timestamp EXPECT (order_status_timestamp > "2021-12-25") ON VIOLATION DROP ROW,
    CONSTRAINT valid_status EXPECT (
      order_status IN ('on the way', 'cancelled', 'return cancelled', 'delivered', 'return processed', 'placed', 'preparing')
    ) 
  )
  COMMENT "Orders with status and timestamp"
  TBLPROPERTIES ("quality" = "silver") 
AS
SELECT
  order_id,
  order_status,
  timestamp(status_timestamp) AS order_status_timestamp
FROM STREAM status_bronze;

-- materialized view
CREATE OR REPLACE MATERIALIZED VIEW full_order_info_gold
  COMMENT "joins order and status tables to see status of any order"
  TBLPROPERTIES ("quality" = "gold")
AS
SELECT
  o.order_id,
  o.order_timestamp,
  s.order_status,
  s.order_status_timestamp
FROM status_silver s
  INNER JOIN orders_silver o
  ON s.order_id = o.order_id;

-- cancelled orders
CREATE OR REFRESH MATERIALIZED VIEW cancelled_orders_gold
  COMMENT "all cancelled orders"
  TBLPROPERTIES ("quality" = "gold")
AS
SELECT
  order_id,
  order_timestamp
  order_status,
  order_status_timestamp,
  date_diff(DAY, order_timestamp, order_status_timestamp) AS days_to_cancel
FROM full_order_info_gold
WHERE order_status = 'cancelled';

-- delivered orders
CREATE OR REFRESH MATERIALIZED VIEW delivered_orders_gold
  COMMENT "all delivered orders"
  TBLPROPERTIES ("quality" = "gold")
AS
SELECT
  order_id,
  order_timestamp
  order_status,
  order_status_timestamp,
  date_diff(DAY, order_timestamp, order_status_timestamp) AS days_to_cancel
FROM full_order_info_gold
WHERE order_status = 'delivered';